In [3]:
 starts = [(16 - min(4, 16)) for (4, 16, 32) in zip([1,2], [3, 5], None)]

SyntaxError: cannot assign to literal (1889556264.py, line 1)

In [4]:
fft_size = [64, 128, 256]
n_modes = [32, 64, 128]
max_modes_list = [48, 96, 192]  # Replace None with actual iterable

starts = [(max_modes - min(size, n_mode)) for (size, n_mode, max_modes) in zip(fft_size, n_modes, max_modes_list)]

print(starts)


[16, 32, 64]


In [7]:
print(list(zip(fft_size, n_modes, max_modes_list)))

[(64, 32, 48), (128, 64, 96), (256, 128, 192)]
